In [1]:
import time
import sys

from graph_nets import blocks
from graph_nets import utils_tf
from graph_nets.demos import models
from graph_nets import modules
from matplotlib import pyplot as plt
from simulate_orbits import *
import numpy as np
from copy import deepcopy

import random as rand

import graph_nets as gn
import sonnet as snt
import tensorflow as tf
import networkx as nx

# For debugging only
#tf.compat.v1.enable_eager_execution()

In [2]:
# Based on the tensorflow_graphics package

def log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def cartesian_to_spherical_coordinates(point_cartesian, eps=None):
    """Function to transform Cartesian coordinates to spherical coordinates.
    This function assumes a right handed coordinate system with `z` pointing up.
    When `x` and `y` are both `0`, the function outputs `0` for `phi`. Note that
    the function is not smooth when `x = y = 0`.
    Note:
      In the following, A1 to An are optional batch dimensions.
    Args:
      point_cartesian: A tensor of shape `[A1, ..., An, 3]`. In the last
        dimension, the data follows the `x`, `y`, `z` order.
      eps: A small `float`, to be added to the denominator. If left as `None`,
        its value is automatically selected using `point_cartesian.dtype`.
      name: A name for this op. Defaults to `cartesian_to_spherical_coordinates`.
    Returns:
      A tensor of shape `[A1, ..., An, 3]`. The last dimensions contains
      (`r`,`theta`,`phi`), where `r` is the sphere radius, `theta` is the polar
      angle and `phi` is the azimuthal angle.
    """
    #with tf.compat.v1.name_scope(name, "cartesian_to_spherical_coordinates",
    #                             [point_cartesian]):
    #  point_cartesian = tf.convert_to_tensor(value=point_cartesian)

    #shape.check_static(
    #    tensor=point_cartesian,
    #    tensor_name="point_cartesian",
    #    has_dim_equals=(-1, 3))

    x, y, z = tf.unstack(point_cartesian, axis=-1)
    radius = tf.norm(tensor=point_cartesian, axis=-1)
    theta = tf.acos(
        tf.clip_by_value(tf.divide(z, radius), -1., 1.))
    phi = tf.atan2(y, x)
    return tf.stack((log10(radius), theta, phi), axis=-1)

def spherical_to_cartesian_coordinates(point_spherical, name=None):
    """Function to transform Cartesian coordinates to spherical coordinates.
    Note:
      In the following, A1 to An are optional batch dimensions.
    Args:
      point_spherical: A tensor of shape `[A1, ..., An, 3]`. The last dimension
        contains r, theta, and phi that respectively correspond to the radius,
        polar angle and azimuthal angle; r must be non-negative.
      name: A name for this op. Defaults to 'spherical_to_cartesian_coordinates'.
    Raises:
      tf.errors.InvalidArgumentError: If r, theta or phi contains out of range
      data.
    Returns:
      A tensor of shape `[A1, ..., An, 3]`, where the last dimension contains the
      cartesian coordinates in x,y,z order.
    """
    #with tf.compat.v1.name_scope(name, "spherical_to_cartesian_coordinates",
    #                           [point_spherical]):
    #point_spherical = tf.convert_to_tensor(value=point_spherical)

    #shape.check_static(
    #    tensor=point_spherical,
    #    tensor_name="point_spherical",
    #    has_dim_equals=(-1, 3))

    logr, theta, phi = tf.unstack(point_spherical, axis=-1)
    r = tf.pow(logr, 10)
    #r = asserts.assert_all_above(r, 0)
    tmp = r * tf.sin(theta)
    x = tmp * tf.cos(phi)
    y = tmp * tf.sin(phi)
    z = r * tf.cos(theta)
    return tf.stack((x, y, z), axis=-1)

In [3]:
def generate_batch_random(planets, x_traj, dp_traj, num_time_steps, batch_size, norm_factor = 1, ellipcity_noise = 0):
    x_traj_ls = []
    p_traj_ls = []
    dp_traj_ls = []
    traj_len = len(x_traj)
    nplanets = len(planets)
    x_batch_np = np.zeros([batch_size, num_time_steps, nplanets, 3])
    dp_batch_np = np.zeros([batch_size, num_time_steps, nplanets, 3])
    t_array = np.random.choice(traj_len-1, batch_size*num_time_steps)
    k = 0
    for i in range(batch_size):
        for j in range(num_time_steps):
            x_batch_np[i,j] = x_traj[t_array[k]]
            dp_batch_np[i,j] = dp_traj[t_array[k]]
            k+=1
            
    x_batch = tf.convert_to_tensor(x_batch_np, dtype=np.float32)
    dp_batch = tf.convert_to_tensor(dp_batch_np, dtype=np.float32)
    return x_batch, dp_batch

def change_orbit_frame(orbits, masses, delta_time):
    x_arr = orbits[:,:3,:]
    v_arr = orbits[:,3:,:]
    p_arr = v_arr[:,:,:]*masses[:,np.newaxis,np.newaxis]
    
    #x_cm_0 = np.sum(x_arr[:,:,0]*masses[:,np.newaxis], axis = 0)/np.sum(masses)
    v_ref = np.sum(p_arr[:,:,:], axis = 0)/np.sum(masses)
    x_cm = np.sum(x_arr[:,:,:]*masses[:,np.newaxis,np.newaxis], axis = 0)/np.sum(masses)
    
    v_new = v_arr - v_ref
    ntime = len(x_arr[0,0])
    t = np.arange(0, ntime*delta_time, delta_time)
    #x_new = x_arr - v_ref*t - x_cm_0[np.newaxis,:,np.newaxis]
    x_new = x_arr - x_cm
    p_new = v_new*masses[:,np.newaxis,np.newaxis]
    
    return x_new.transpose(2,0,1), p_new.transpose(2,0,1)

def convert_array_to_polar(v):
    r = tf.norm(v)
    phi = tf.atan2(v[1],v[0])
    theta = tf.math.acos(v[2],r)
    return tf.stack((tf.log(r), phi, theta))

def get_input_graph(masses, xtraj, t, noise_level = 0.0, symmetric = True, polar = False):
    '''
    Convert a given time into a GraphNets graph that can be used to train a model
    
    Parameters
    ----------
    planets : ls
        A list of planets using the Body class
    xtraj: np.array([num_time_steps, nplanets, 2])
        The positions and momenta of the trajectory for each planet
    t: int
        The time at which the trajectory is evaluated
    noise_level: float
        Fractional noise added to the training, defaults to 0.05
        
    Returns
    -------
    graph_dict: dict
        A dictionary containing globals, edges, nodes, senders and receivers
    '''
    nplanets = len(planets)
    nodes, edges, senders, receivers = [], [], [], []
    for i in range(nplanets):
        noise_mass = np.random.normal(0, noise_level)
        #mass = planets[i].mass#*(1+noise_mass)
        #nodes.append([mass]) #Use Mercury's mass for units to normalize
        nodes.append([masses[i]]) #Use Mercury's mass for units to normalize
        for j in range(nplanets):
            # I do this instead of if i != j, so the distances and forces are not duplicate, this
            # improves the model. I am basically telling the model that F(ij)=F(ji)
            if symmetric == True:
                if i > j:
                    if planets[j].name == 'dummy':
                        pass
                    else:
                        #print(planets[i].name, planets[j].name)
                        d = xtraj[t,j,:] - xtraj[t,i,:]
                        if polar == True: 
                            d = cartesian_to_spherical_coordinates(d)
                        if noise_level > 0:
                            noise_dist = tf.random.normal([3], 0, noise_level, tf.float32) 
                            edges.append(d*(1+noise_dist))
                        else:
                            edges.append(d)
 
                        receivers.append(i)
                        senders.append(j)
            else:
                if i != j:
                    d = xtraj[t,j,:] - xtraj[t,i,:]
                    if noise_level > 0:
                        noise_dist = tf.random.normal([3], 0, noise_level, tf.float32) 
                        edges.append(d*(1+noise_dist))
                    else:
                        edges.append(d)
 
                    receivers.append(i)
                    senders.append(j)

    
    return{
      "globals": [G],
      "nodes": nodes,
      "edges": edges, 
      "receivers": receivers, 
      "senders": senders 
    }  

def get_momentum_update(output_op, symmetric = True):
    reducer = tf.unsorted_segment_sum
    
    b1 = blocks.ReceivedEdgesToNodesAggregator(reducer=reducer)(output_op)
    b2 = blocks.SentEdgesToNodesAggregator(reducer=reducer)(output_op)
    if symmetric == True:
        dp = b1-b2
    else:
        dp = b1
    return dp

def build_rotation_matrix(a,b,g):
    A0 = tf.stack([tf.cos(a)*tf.cos(b), tf.sin(a)*tf.cos(b), -tf.sin(b)], 
                  axis=0)
    A1 = tf.stack([tf.cos(a)*tf.sin(b)*tf.sin(g)-tf.sin(a)*tf.cos(g), 
                   tf.sin(a)*tf.sin(b)*tf.sin(g)+tf.cos(a)*tf.cos(g),
                   tf.cos(b)*tf.sin(g)], axis=0)
    A2 = tf.stack([tf.cos(a)*tf.sin(b)*tf.cos(g)+tf.sin(a)*tf.sin(g), 
                   tf.sin(a)*tf.sin(b)*tf.cos(g)-tf.cos(a)*tf.sin(g),
                   tf.cos(b)*tf.cos(g)], axis=0)
    
    return tf.stack((A0, A1, A2), axis=1)

def build_rotation_matrix_np(a,b,g):
    A0 = np.stack([np.cos(a)*np.cos(b), np.sin(a)*np.cos(b), -np.sin(b)], 
                  axis=0)
    A1 = np.stack([np.cos(a)*np.sin(b)*np.sin(g)-np.sin(a)*np.cos(g), 
                   np.sin(a)*np.sin(b)*np.sin(g)+np.cos(a)*np.cos(g),
                   np.cos(b)*np.sin(g)], axis=0)
    A2 = np.stack([np.cos(a)*np.sin(b)*np.cos(g)+np.sin(a)*np.sin(g), 
                   np.sin(a)*np.sin(b)*np.cos(g)-np.cos(a)*np.sin(g),
                   np.cos(b)*np.cos(g)], axis=0)
    
    return np.stack((A0, A1, A2), axis=1)

def convert_graph_to_cartesian(graph):
    x = np.exp(graph.edges[:,0])*tf.sin(graph.edges[:,2])*tf.cos(graph.edges[:,1])
    y = np.exp(graph.edges[:,0])*tf.sin(graph.edges[:,2])*tf.sin(graph.edges[:,1])
    z = np.exp(graph.edges[:,0])*tf.cos(graph.edges[:,2])
    newgraph = graph.replace(edges=tf.stack((x,y,z), axis = 1))

    return newgraph

## Integration functions

In [4]:
def get_distances(x, input_graph):
    '''
    Convert positions to distances
    '''
    new_graph = input_graph.replace(nodes = x)
    e1 = blocks.broadcast_sender_nodes_to_edges(new_graph)
    e2 = blocks.broadcast_receiver_nodes_to_edges(new_graph)
    dx = e1 - e2
    return dx

def model_gn(dx, input_graph, normalization = 1, symmetric = True):
    '''
    Use the model to go from dx to dp
    '''
    graph = input_graph.replace(edges = dx)
    outputs = model(graph)
    dp_unnorm = get_momentum_update(outputs, symmetric)
    dp = dp_unnorm*normalization

    forces_unnorm = outputs.edges
    forces = forces_unnorm*normalization
    return dp, forces

def get_leapfrog_step(x0, ph, delta_time, input_graph, model, normalization = 1, symmetric = True):
    
    delta_x = delta_time/input_graph.nodes
    
    x1 = x0 + ph*delta_x
    
    dx = get_distances(x0, input_graph)
    
    dp, force = model(dx, input_graph, normalization, symmetric)
    ph3 = ph + dp

    return x1, ph3, dp, force

def leapfrog_integration(x0, p0, delta_time, 
                         input_graph, num_steps, model, 
                         normalization = 1, symmetric = True):
    '''
    Learn the orbit through leapfrom integration
    '''
    def body(i, x0, p0, x_pred, dp_pred, f_pred):
        x, ph, dp, force =  get_leapfrog_step(
            x0, p0, delta_time, 
            input_graph, 
            model, 
            normalization,
            symmetric, )
        return i+1, x, ph, x_pred.write(i, x), dp_pred.write(i-1, dp/normalization), f_pred.write(i-1, force)
    
    # Distance
    dx = get_distances(x0, input_graph)

    # Model predict*norm_p = F*dt
    # (ph = phalf)
    dph, fh = model(dx, input_graph, normalization, symmetric)
    ph = p0 + 0.5*dph
    x = tf.identity(x0)
    
    i = 0

    x_pred = tf.TensorArray(
      dtype=tf.float32, size=num_steps, element_shape=x0.shape)
    dp_pred = tf.TensorArray(
      dtype=tf.float32, size=num_steps-1, element_shape=x0.shape)
    f_pred = tf.TensorArray(
      dtype=tf.float32, size=num_steps-1, element_shape=fh.shape)

    x_pred = x_pred.write(0, x0)
    
    _, _, _, x_pred, dp_pred, f_pred = tf.while_loop(
    lambda i, *unused_args: i < num_steps,
        body,
        loop_vars = [1, x0, ph, x_pred, dp_pred, f_pred]
    )
    return x_pred.stack(), dp_pred.stack(), f_pred.stack()

In [5]:
# Global constants
DAY = 24*3600. # Day in seconds
YEAR = 365.25*DAY #Year
delta_time = (2/24.)*DAY/YEAR # 1 hour
#delta_time = (2/24.) # Planets have units of days, not years!
#delta_time = 0.1*DAY/YEAR # 1 hour

symmetric = True

total_time_traj = 20 #Years
num_time_steps_total = int(total_time_traj/delta_time)
num_time_steps_test = 10000
num_time_steps_test_int = 5000

batch_size_tr = 75 # This way an 'Epoch' (when it has gone through all the points) is approximately 1000
#batch_size_tr = 50
#total_time_tr = 1000*delta_time
num_time_steps_tr = int(total_time_traj/delta_time) - num_time_steps_test

patience = 10
#d_patience = 1e-5
d_patience = 0
noise_level = 0.05

# How much time between logging and printing the current results.
log_every_iterations = 1000
num_training_iterations = 200000

sun = Body()
sun.name = 'Sun'
sun.mass = 1 # solar masses
#sun.mass = 1/(0.33011 * 10**24/MSUN)

mercury = Body()
mercury.name = 'Mercury'
mercury.mass = 0.33011 * 10**24/MSUN # Earth masses
#mercury.mass = 1 # Earth masses

venus = Body()
venus.name = 'Venus'
venus.mass = 4.8685 * 10**24/MSUN # Earth masses

earth = Body()
earth.name = 'Earth'
earth.mass =MEARTH/MSUN # Earth masses

mars = Body()
mars.name = 'Mars'
mars.mass =0.642 * 10**24/MSUN # Earth masses

jupiter = Body()
jupiter.name = 'Jupiter'
jupiter.mass =1898.19 * 10**24/MSUN # Earth masses

io = Body()
io.name = 'Io'
io.mass =0.08931900 * 10**24/MSUN # Earth masses

europa = Body()
europa.name = 'Europa'
europa.mass =0.048 * 10**24/MSUN # Earth masses

ganymede = Body()
ganymede.name = 'Ganymede'
ganymede.mass = 0.14819 * 10**24/MSUN # Earth masses

callisto = Body()
callisto.name = 'Callisto'
callisto.mass =0.10759 * 10**24/MSUN # Earth masses

saturn = Body()
saturn.name = 'Saturn'
saturn.mass =568 * 10**24/MSUN # Earth masses

dione = Body()
dione.name = 'Dione'
dione.mass =0.0011 * 10**24/MSUN # Earth masses

rhea = Body()
rhea.name = 'Rhea'
rhea.mass =0.0023 * 10**24/MSUN # Earth masses

titan = Body()
titan.name = 'Titan'
titan.mass =0.135 * 10**24/MSUN # Earth masses

uranus = Body()
uranus.name = 'Uranus'
uranus.mass =86.8 * 10**24/MSUN # Earth masses

neptune = Body()
neptune.name = 'Neptune'
neptune.mass =102 * 10**24/MSUN # Earth masses



orbit_sun = np.loadtxt('nasa_orbits/sun_center/sun.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_mercury = np.loadtxt('nasa_orbits/sun_center/mercury.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_venus = np.loadtxt('nasa_orbits/sun_center/venus.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_earth = np.loadtxt('nasa_orbits/sun_center/earth.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_mars = np.loadtxt('nasa_orbits/sun_center/mars.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_jupiter = np.loadtxt('nasa_orbits/sun_center/jupiter.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_io = np.loadtxt('nasa_orbits/sun_center/io.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_europa = np.loadtxt('nasa_orbits/sun_center/europa.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_ganymede = np.loadtxt('nasa_orbits/sun_center/ganymede.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_callisto = np.loadtxt('nasa_orbits/sun_center/callisto.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_saturn = np.loadtxt('nasa_orbits/sun_center/saturn.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_dione = np.loadtxt('nasa_orbits/sun_center/dione.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_rhea = np.loadtxt('nasa_orbits/sun_center/rhea.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_titan = np.loadtxt('nasa_orbits/sun_center/titan.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_uranus = np.loadtxt('nasa_orbits/sun_center/uranus.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')
orbit_neptune = np.loadtxt('nasa_orbits/sun_center/neptune.txt', usecols = [2,3,4, 5, 6, 7], unpack=True, delimiter=',')

masses = np.array([sun.mass, mercury.mass, venus.mass, earth.mass, mars.mass,
                   jupiter.mass + io.mass + europa.mass + ganymede.mass + callisto.mass, 
                   saturn.mass + dione.mass + rhea.mass + titan.mass, 
                   uranus.mass, neptune.mass
                  ])


orbit_jupiter[:,:] += (orbit_io[:,:]*io.mass + orbit_europa[:,:]*europa.mass + 
                       orbit_ganymede[:,:]*ganymede.mass + orbit_callisto[:,:]*callisto.mass)/jupiter.mass


orbit_saturn[:,:] += (orbit_dione[:,:]*dione.mass + orbit_rhea[:,:]*rhea.mass + 
                       orbit_titan[:,:]*titan.mass)/saturn.mass

orbits_orig = np.stack([orbit_sun, orbit_mercury, orbit_venus, orbit_earth, orbit_mars,
                   orbit_jupiter, orbit_saturn, orbit_uranus, orbit_neptune
                       ])

orbits_orig[:,3:,:] *= 365.25

jupiter.mass = masses[5]
saturn.mass = masses[6]

x_traj_np, p_traj_np = change_orbit_frame(orbits_orig[:7], masses[:7], delta_time)


sun.pos = x_traj_np[0,:,0]
mercury.pos = x_traj_np[1,:,0]
venus.pos = x_traj_np[2,:,0]
earth.pos = x_traj_np[3,:,0]
mars.pos =x_traj_np[4,:,0]
jupiter.pos = x_traj_np[5,:,0]
io.pos = x_traj_np[6,:,0]
europa.pos = x_traj_np[7,:,0]
ganymede.pos = x_traj_np[8,:,0]
callisto.pos = x_traj_np[9,:,0]
saturn.pos = x_traj_np[6,:,0]
uranus.pos = x_traj_np[7,:,0]
neptune.pos = x_traj_np[8,:,0]

sun.mom = p_traj_np[0,:,0]
mercury.mom = p_traj_np[1,:,0]
venus.mom = p_traj_np[2,:,0]
earth.mom = p_traj_np[3,:,0]
mars.mom =p_traj_np[4,:,0]
jupiter.mom = p_traj_np[5,:,0]
io.mom = p_traj_np[6,:,0]
europa.mom = p_traj_np[7,:,0]
ganymede.mom = p_traj_np[8,:,0]
callisto.mom = p_traj_np[9,:,0]
saturn.mom = p_traj_np[6,:,0]
uranus.mom = p_traj_np[7,:,0]
neptune.mom = p_traj_np[8,:,0]

planets = [sun, mercury,
           venus, 
           earth, mars, 
           jupiter,
           saturn, 
           #uranus, neptune
          ]

nplanets = len(planets)
logmasses_ini = -6*np.ones(nplanets)
logmasses_ini[0] = -0.5
#mean_logmasses_ini = tf.convert_to_tensor(np.random.uniform(low=-7.0, high=0.0, size=nplanets),dtype = np.float32)
mean_logmasses_ini = tf.convert_to_tensor(logmasses_ini,dtype = np.float32)
#mean_logmasses_ini = tf.convert_to_tensor(-2*np.ones(nplanets),dtype = np.float32)
#mean_logmasses_ini = tf.convert_to_tensor(np.log(masses[:nplanets]/earth.mass),dtype = np.float32)
delta_logmasses = tf.random.normal([nplanets], 0, 0.1, tf.float32)
mean_logmasses_ini = mean_logmasses_ini+delta_logmasses
sigma_logmasses_ini = tf.convert_to_tensor(np.ones(nplanets),dtype = np.float32)
#masses_tr = tf.convert_to_tensor(np.log(masses[:nplanets]),dtype = np.float32)

planets_gn = deepcopy(planets)

dp_traj_np = p_traj_np[1:] - p_traj_np[:-1]
dv_traj_np = dp_traj_np/masses[np.newaxis, :nplanets, np.newaxis]

p_norm = np.std(dp_traj_np)
v_norm = np.std(dv_traj_np)
#p_norm = 1

x_traj_test = tf.convert_to_tensor(x_traj_np[:num_time_steps_test], dtype=np.float32)
p_traj_test = tf.convert_to_tensor(p_traj_np[:num_time_steps_test], dtype=np.float32)
dp_traj_norm = tf.convert_to_tensor(
    dp_traj_np[:num_time_steps_test-1]/p_norm, dtype=np.float32)
dv_traj_norm = tf.convert_to_tensor(
    dv_traj_np[:num_time_steps_test-1]/v_norm, dtype=np.float32)



x_traj_np_tr = x_traj_np[num_time_steps_test:]
dp_traj_np_tr = dp_traj_np[num_time_steps_test:]/p_norm
dv_traj_np_tr = dv_traj_np[num_time_steps_test:]/v_norm


#dv_traj_np_tr[:,0]*=1e7
#x_traj_np_tr[:,0]*=1e7

input_dict_test = get_input_graph(masses, x_traj_test, 0, noise_level = 0.0, symmetric = symmetric)

input_graph_test = utils_tf.data_dicts_to_graphs_tuple([input_dict_test])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
np.log10(masses)

array([ 0.        , -6.77995863, -5.61122214, -5.52233756, -6.49108229,
       -3.0201877 , -3.54416317, -4.36009759, -4.29001714])

In [7]:
# Create the graph network.
graph_net_module = blocks.EdgeBlock(
        #edge_model_fn=lambda: snt.nets.MLP([128, 128, 128, 128, 3]),
    #edge_model_fn=lambda: snt.Linear(output_size=3),
    edge_model_fn=lambda: snt.nets.MLP([32, 32, 3]), 
    use_edges = True,
    use_receiver_nodes = True,
    use_sender_nodes = True,
    use_globals = False,
    )

In [8]:
x_traj_tr, dv_traj_tr = generate_batch_random(planets, x_traj_np_tr, dv_traj_np_tr, num_time_steps=num_time_steps_tr, batch_size=batch_size_tr)

t = tf.random_uniform([], minval=0, maxval=num_time_steps_tr - 1, dtype=tf.int32)

# I think the maxes should be 2pi, pi, pi
alpha = tf.random_uniform([], minval=0, maxval=2*np.pi, dtype=tf.dtypes.float32)
beta = tf.random_uniform([], minval=0, maxval=np.pi, dtype=tf.dtypes.float32)
gamma = tf.random_uniform([], minval=0, maxval=np.pi, dtype=tf.dtypes.float32)
R = build_rotation_matrix(alpha,beta,gamma)

input_dict_tr = []
x_traj_tr_rot = tf.einsum('ijkl,lm->ijkm', x_traj_tr,R)
#x_traj_tr_rot = x_traj_tr

mean_logmasses_tr = tf.Variable(mean_logmasses_ini, trainable = True)
#sigma_logmasses_tr = tf.Variable(sigma_logmasses_ini, trainable = True)
#rand_tr = tf.random.normal([nplanets], 0, 1, tf.float32)
#logmasses_tr = mean_logmasses_tr + sigma_logmasses_tr*rand_tr
logmasses_tr = mean_logmasses_tr

#masses_tf = tf.convert_to_tensor(np.log(masses[:nplanets]/mercury.mass), dtype=tf.dtypes.float32)
for i in range(batch_size_tr):
    input_dict_tr.append(get_input_graph(logmasses_tr, x_traj_tr_rot[i], t, noise_level = noise_level, symmetric=symmetric, polar = True))

    
input_pos_tr = x_traj_tr_rot[:,t]
input_graph_tr = utils_tf.data_dicts_to_graphs_tuple(input_dict_tr)
#logmass_graph_tr = graph_net_module_1(input_graph_tr)
#mass_graph_tr = logmass_graph_tr.replace(nodes = tf.exp(logmass_graph_tr.nodes))

#new_graph = average_masses(output_model, batch_size_tr, nplanets)
#input_graph_tr = input_graph_tr.replace(nodes = new_graph.nodes)
forces_graph_tr_sph = graph_net_module(input_graph_tr)
forces_graph_tr = forces_graph_tr_sph.replace(edges = spherical_to_cartesian_coordinates(forces_graph_tr_sph.edges))

b1_tr = blocks.ReceivedEdgesToNodesAggregator(reducer = tf.unsorted_segment_sum)(forces_graph_tr)
b2_tr = blocks.SentEdgesToNodesAggregator(reducer = tf.unsorted_segment_sum)(forces_graph_tr)
summed_forces_tr = b1_tr-b2_tr

#newnodes_tr = tf.concat([forces_graph_tr.nodes, summed_forces_tr], axis = 1)

#summed_forces_graph_tr = forces_graph_tr.replace(nodes=newnodes_tr)

#acceleration_graph_tr = graph_net_module_3(summed_forces_graph_tr)

acceleration_tr = tf.divide(summed_forces_tr, tf.pow(forces_graph_tr.nodes, 10))
output_ops_tr = tf.reshape(acceleration_tr, shape=[batch_size_tr, nplanets, 3])
#output_ops_tr = tf.reshape(acceleration_graph_tr.nodes, shape=[batch_size_tr, nplanets, 3])
target_nodes_tr = tf.einsum('ijk,kl->ijl', dv_traj_tr[:,t],R)

norm_tf = tf.reduce_mean(tf.norm(target_nodes_tr, axis = 2,  keep_dims=True),axis=0)
dp_tr = (output_ops_tr - target_nodes_tr)/norm_tf*tf.reduce_mean(norm_tf)
loss_op_tr = tf.reduce_mean(
    tf.reduce_sum(
    (dp_tr)**2., axis = -1
    ))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
def force_newton(x, m1, m2):
    return G*m1*m2/np.linalg.norm(x, keepdims=True)**3.*x

nforce = 200
rand_force = np.random.choice(np.arange(0,num_time_steps_test), nforce, replace=False)
rand_angle_1 = np.random.uniform(low = 0, high = 2*np.pi, size = nforce)
rand_angle_2 = np.random.uniform(low = 0, high = np.pi, size = nforce)
rand_angle_3 = np.random.uniform(low = 0, high = np.pi, size = nforce)
positions = np.empty([nforce, nplanets, 3])
dv_val_np_true = np.empty([nforce, nplanets, 3])
forces_true = np.empty([nplanets - 1, nforce, 3])
planets_forces = deepcopy(planets)
for i in range(nforce):
    j = rand_force[i]
    rot_mat = build_rotation_matrix_np(rand_angle_1[i],
                                      rand_angle_2[i],
                                      rand_angle_3[i])
    positions[i,:,:] = x_traj_np[j].dot(rot_mat)
    dv_val_np_true[i,:,:] = dv_traj_np[j].dot(rot_mat)
    for p in range(nplanets):
        if p > 0: 
            forces_true[p-1,i,:] = force_newton(positions[i,p,:] - positions[i,0,:], 1, planets_forces[p].mass)

        #if p == 0:
            #positions[i,p,:] = np.zeros(planets_forces[0].pos[:3].shape)
        #else:
            #positions[i,p,:] = (x_traj_np[j,p] - x_traj_np[j,0]).dot(rot_mat)
            #forces_true[p-1,i,:] = force_newton(positions[i,p,:], 1, planets_forces[p].mass)


x_force = tf.convert_to_tensor(positions, dtype = np.float32)
dv_val_true = tf.convert_to_tensor(dv_val_np_true/v_norm, dtype = np.float32)
input_dict_val = []
for i in range(nforce):
    input_dict_val.append(get_input_graph(logmasses_tr, x_force, i, noise_level = noise_level, polar = True))

input_graph_val = utils_tf.data_dicts_to_graphs_tuple(input_dict_val)
#logmass_graph_val = graph_net_module_1(input_graph_val)
#mass_graph_val = logmass_graph_val.replace(nodes = tf.exp(logmass_graph_val.nodes))

#new_graph = average_masses(output_model, batch_size_tr, nplanets)
#input_graph_tr = input_graph_tr.replace(nodes = new_graph.nodes)
forces_graph_val_sph = graph_net_module(input_graph_val)
forces_graph_val = forces_graph_val_sph.replace(edges=spherical_to_cartesian_coordinates(forces_graph_val_sph.edges))

b1_val = blocks.ReceivedEdgesToNodesAggregator(reducer = tf.unsorted_segment_sum)(forces_graph_val)
b2_val = blocks.SentEdgesToNodesAggregator(reducer = tf.unsorted_segment_sum)(forces_graph_val)
summed_forces_val = b1_val-b2_val
#newnodes_val = tf.concat([forces_graph_val.nodes, summed_forces_val], axis = 1)

#summed_forces_graph_val = forces_graph_tr.replace(nodes=newnodes_val)

#acceleration_graph_val = graph_net_module_3(summed_forces_graph_val)
acceleration_val = tf.divide(summed_forces_val, tf.pow(forces_graph_val.nodes, 10))

#output_ops_tr = get_momentum_update(output_model, symmetric = symmetric)
 
dv_val_pred = tf.reshape(acceleration_val, shape=[nforce, nplanets, 3])

force_val = tf.reshape(forces_graph_val.edges, shape=[nforce, nplanets*(nplanets - 1)//2, 3])

dp_val = (dv_val_pred - dv_val_true)/norm_tf*tf.reduce_mean(norm_tf)
loss_val = tf.reduce_mean(
    tf.reduce_sum(
    (dp_val)**2., axis = -1
    ))

Learning rate makes a HUGE difference. If it is oscillating around a certain number (e.g. 1.4), it needs to be lower

In [10]:
'''
initial_learning_rate = 0.1
decay_steps = 1.0
decay_rate = 0.5
learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(
  initial_learning_rate, decay_steps, decay_rate)

# Define configuration parameters
start_lr = 0.01
min_lr = 0.00001
max_lr = 0.1
rampup_epochs = 10
sustain_epochs = 0
exp_decay = 0.8

# Define the scheduling function
def schedule(epoch):
    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        if epoch < rampup_epochs:
            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        else:
            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)
'''

'\ninitial_learning_rate = 0.1\ndecay_steps = 1.0\ndecay_rate = 0.5\nlearning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(\n  initial_learning_rate, decay_steps, decay_rate)\n\n# Define configuration parameters\nstart_lr = 0.01\nmin_lr = 0.00001\nmax_lr = 0.1\nrampup_epochs = 10\nsustain_epochs = 0\nexp_decay = 0.8\n\n# Define the scheduling function\ndef schedule(epoch):\n    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):\n        if epoch < rampup_epochs:\n            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr\n        elif epoch < rampup_epochs + sustain_epochs:\n            lr = max_lr\n        else:\n            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr\n        return lr\n    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)\n'

In [11]:
# Optimizer.
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate)#, amsgrad=True)
step_op = optimizer.minimize(loss_op_tr)

In [12]:
# This cell resets the Tensorflow session, but keeps the same computational
# graph.

try:
    sess.close()
except NameError:
    pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

nsteps_no_improvement = 0
last_iteration = 0
best_loss = 1e100
iterations = []
losses_tr = []
losses_val = []
losses_test = []

In [13]:
logmasses_tr.eval(session=sess)

array([-0.49256682, -6.128961  , -5.9554133 , -6.143643  , -6.1051598 ,
       -6.070259  , -6.038288  ], dtype=float32)

In [ ]:
start_time = time.time()
last_log_time = start_time

print("# (iteration number), T (elapsed seconds), "
      "Ltr (training 1-step loss), Ltest (test loss)")

for iteration in range(last_iteration, num_training_iterations):
    last_iteration = iteration
    train_values = sess.run({
        "step": step_op,
        "loss": loss_op_tr,
        "input_pos": input_pos_tr,
        "input_graph": input_graph_tr,
        "target_nodes": target_nodes_tr,
        "outputs": output_ops_tr,
        "mean_logmasses": logmasses_tr,
        #"sigma_logmasses": sigma_logmasses_tr,
    })
    
    the_time = time.time()
    
    if iteration%log_every_iterations == 0:
        val_values = sess.run({
            "output": dv_val_pred,
            "target": dv_val_true,
            "force": force_val,
            "summed_forces": summed_forces_val,
            "loss": loss_val,
            })
        #test_orbit = sess.run({
        #    "x_pred": xp, 
        #    "dp_pred": pp,
        #    "f_pred": output_force,
        #    "loss": loss_test
        #})

        #learning_rate = one_cycle_lr(iteration, learning_rate, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)
        #optimizer = tf.train.AdamOptimizer(learning_rate)
        
        # Implement early stopping
        if val_values["loss"] > best_loss: 
            nsteps_no_improvement += 1
        else: 
            save_sess= sess # save session
            nsteps_no_improvement = 0
            best_loss = val_values["loss"]
            
        # If we have reached early stopping, stop the training
        if nsteps_no_improvement >= patience: 
            train_values = save_sess.run({
                "step": step_op,
                "loss": loss_op_tr,
                "input_pos": input_pos_tr,
                "input_graph": input_graph_tr,
                "target_nodes": target_nodes_tr,
                "outputs": output_ops_tr,
                "mean_logmasses": logmasses_tr,
            })

            val_values = save_sess.run({
                "output": dv_val_pred,
                "target": dv_val_true,
                "force": force_val,
                "summed_forces": summed_forces_val,
                "loss": loss_val,
            })
            
            print('Convergence achieved')
            print('Iterations = ', iteration)
            print('Time = ', time.time() - start_time, 'seconds.')
            print('Training 1-step loss = ', train_values["loss"])
            break
    
            
        last_log_time = the_time
        elapsed = time.time() - start_time
        iterations.append(iteration)
        losses_tr.append(train_values["loss"])
        losses_val.append(val_values["loss"])
        #losses_test.append(test_orbit["loss"])
        #print(
        #    "# {:06d}, T {:.1f}, Ltr {:.6f}, Lval {:.6f}, Ltest {:.6f}".format(
        #        iteration,  elapsed, train_values["loss"], train_values["val_loss"], test_orbit["loss"]))
        
        print(
            "# {:06d}, T {:.1f}, Ltr {:.6f}, Lval {:.6f} , Msun {:.6f} , Mmerc {:.6f}, ".format(
                iteration,  elapsed, train_values["loss"], val_values["loss"], 
            train_values["mean_logmasses"][0], train_values["mean_logmasses"][1]
            ))

# (iteration number), T (elapsed seconds), Ltr (training 1-step loss), Ltest (test loss)
# 000000, T 140.0, Ltr 79885498352402432.000000, Lval 6699404341805056.000000 , Msun -0.493567 , Mmerc -6.127961, 
# 001000, T 375.3, Ltr 87745480.000000, Lval 155455856.000000 , Msun -0.498893 , Mmerc -6.122274, 
# 002000, T 612.7, Ltr 2580.005859, Lval 7069.189453 , Msun -0.498893 , Mmerc -6.122274, 
# 003000, T 848.1, Ltr 223.450104, Lval 1366.173340 , Msun -0.498893 , Mmerc -6.122274, 
# 004000, T 1090.1, Ltr 5.010312, Lval 634.778381 , Msun -0.498893 , Mmerc -6.122274, 
# 005000, T 1338.6, Ltr 1346.288818, Lval 353.745758 , Msun -0.498893 , Mmerc -6.122274, 
# 006000, T 1584.6, Ltr 10.871880, Lval 199.674683 , Msun -0.498893 , Mmerc -6.122274, 
# 007000, T 1830.6, Ltr 12.210464, Lval 112.340790 , Msun -0.498893 , Mmerc -6.122274, 
# 008000, T 2075.8, Ltr 2.788873, Lval 57.140526 , Msun -0.498893 , Mmerc -6.122274, 
# 009000, T 2319.2, Ltr 1.023803, Lval 65.409706 , Msun -0.498893 , Mmerc -6.12

In [ ]:
#save_path = saver.save(sess, "./model.ckpt")
#print("Model saved in path: %s" % save_path)

In [ ]:
tr = train_values["target_nodes"]
ou = train_values["outputs"]
#masses_learned = train_values["masses"]

In [ ]:
fig = plt.figure()
for i in range(nplanets):
    plt.scatter(tr[:,i,0], tr[:,i,1])

In [ ]:
fig = plt.figure()
for i in range(nplanets):
    plt.scatter(ou[:,i,0], ou[:,i,1])

In [ ]:
val_pred = val_values["output"]
val_truth = val_values["target"]

In [ ]:
fig = plt.figure()
for i in range(nplanets):
    plt.scatter(val_pred[:,i,0], val_pred[:,i,1])

In [ ]:
fig = plt.figure()
for i in range(nplanets):
    plt.scatter(val_truth[:,i,0], val_truth[:,i,1])

In [ ]:
val_force = val_values["force"]
val_masses = 10**(train_values["mean_logmasses"])

In [ ]:
indices = [0,1,3,6,10,15,21]
X = np.zeros([(nplanets-1)*nforce,5])
F = np.zeros([(nplanets-1)*nforce,3])
F_norm = np.mean(val_force)
for i in range(nplanets-1):
    #print( X[i*nforce:(i+1)*nforce].shape, positions[:,i+1,:].shape)
    X[i*nforce:(i+1)*nforce,0] = val_masses[i+1]
    X[i*nforce:(i+1)*nforce,1:4] = positions[:,i+1,:]
    X[i*nforce:(i+1)*nforce,4] = np.linalg.norm(positions[:,i+1,:], axis = -1)#**3
    #F[i*nforce:(i+1)*nforce,:] = forces_true[i,:,:]#/F_norm
    F[i*nforce:(i+1)*nforce,:] = val_force[:,indices[i],:]/F_norm

In [ ]:
from pysr import pysr
# Learn equations
equations_y = pysr(X[:,:5], F[:,1], niterations=100,
            binary_operators=["mult", "div"],
            unary_operators=["square", "cube", "sqrtm"])

In [ ]:
equations_x

In [ ]:
equations_z

In [ ]:
-np.arange(nplanets)

In [ ]:
mean_logmasses_ini.eval(session=sess)